# CE-40719: Deep Learning
## HW3 - CNN / CNN Case Studies / CNN Applications
(18 points)

#### Name: Sadroddin Barikbin
#### Student No.: 98208824

In this exercise we are going to implement a simple residual netwrok architecture to classify images from Cifar10 dataset. Here we give you a suggestion for architecture but you are allowed to  make changes and experiment to get better results. Explain your ideas or reference the papers that you take the ideas from. You are allowed to use out-of-the-box pytorch modules in `torch.nn`.

## Architecture

_All convolution layers have `3 * 3` kernel,  `padding=1`, batch normalization and relu activation_

__ResidualBlock:__ _in_channel, out_channel, stride_

- Conv(in_channel, out_channel, stride)
- Conv(out_channel, out_channel, stride=1)

`*` This block has a residual connection. To match dimmensions of output and residual use `1 * 1` convolution and stride. 

__ResidualLayer:__ _in_channel, out_channel, stride_

- ResidualBlock(in_channel, out_channel, stride)
- ResidualBlock(out_channel, out_channel, stride=1)

__ResidualNetwork__:
- Conv(3, 64, stride=1)
- ResidualLayer(64, 64, stride=1)
- ResidualLayer(64, 128, stride=2)
- ResidualLayer(128, 256, stride=2)
- ResidualLayer(256, 512, stride=2)
- AveragePool(4, 4)
- Linear(512, 10)


In [0]:
import torch
from torch import nn,optim
from torch.nn import functional as F
import torchvision
import torchvision.transforms as transforms

In [0]:
class ResidualBlock(nn.Module):
  def __init__(self,in_channel,out_channel,stride):
    super(ResidualBlock,self).__init__()
    self.conv1=nn.Conv2d(in_channel,out_channel,3,stride=stride,padding=1)
    self.batch1=nn.BatchNorm2d(out_channel)
    self.conv2=nn.Conv2d(out_channel,out_channel,3,padding=1)
    self.batch2=nn.BatchNorm2d(out_channel)
    self.reshaper=nn.Conv2d(in_channel,out_channel,1,stride=stride)
  def forward(self,x):
    y=F.relu(self.batch1(self.conv1(x)))
    y=F.relu(self.batch2(self.conv2(y)))
    return self.reshaper(x)+y



In [0]:
class ResidualLayer(nn.Module):
  def __init__(self,in_channel,out_channel,stride):
    super(ResidualLayer,self).__init__()
    self.block1=ResidualBlock(in_channel,out_channel,stride)
    self.block2=ResidualBlock(out_channel,out_channel,1)
  def forward(self,x):
    return self.block2(self.block1(x))

In [0]:
class ResidualNetwork(nn.Module):
    def __init__(self):
        super(ResidualNetwork, self).__init__()
        self.conv=nn.Conv2d(3,64,3,padding=1)
        self.res1=ResidualLayer(64,64,1)
        self.res2=ResidualLayer(64,128,2)
        self.res3=ResidualLayer(128,256,2)
        self.res4=ResidualLayer(256,512,2)
        self.pool=nn.AvgPool2d(4)
        self.line=nn.Linear(512,10)
    def forward(self, x):
        return self.line(torch.flatten(self.pool(self.res4(self.res3(self.res2(self.res1(self.conv(x)))))),1))

In [34]:
batch_size_train = 256
batch_size_test = 256

transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size_train)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)
testloader=torch.utils.data.DataLoader(testset, batch_size=batch_size_test)
classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [19]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [0]:
num_epoches = 20
model=ResidualNetwork()
model.to(device)
#optimizer=optim.Adam(model.parameters(),lr=0.1,weight_decay=0.3)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [40]:
for epoch in range(num_epoches):
        print('EPOCH {:2d}:'.format(epoch + 1))
        running_loss = 0.
        for i, (x, y) in enumerate(trainloader):
            inputs, labels = x.to(device), y.to(device)
            optimizer.zero_grad()
            out= model(inputs)
            loss = criterion(out, labels)
            loss.backward()
            optimizer.step()  
            running_loss += loss.item()
            if i % 100 == 99:
                test_loss = 0.
                with torch.no_grad():
                    for data in testloader:
                        images, labels = data[0].to(device), data[1].to(device)
                        out = model(images)
                        loss = F.cross_entropy(out, labels)
                        test_loss+=loss.item()
                print('\titeration {:4d}: training_loss = {:5f}, test_loss = {:5f}'.format(i + 1, running_loss/100, test_loss/len(testloader)))
                running_loss = 0.
        with torch.no_grad():
            correct = 0
            total = 0
            class_correct = [0.] * 10
            class_total = [0.] * 10
            for data in testloader:
                images, labels = data[0].to(device), data[1].to(device)
                out= model(images)
                _, predicted = torch.max(out.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()                
                for i in range(len(labels)):
                    class_total[labels[i]]+=1
                    if predicted[i]==labels[i]:
                        class_correct[labels[i]]+=1
        print('test_accuracy = {:5f}'.format(correct / total))
        for i in range(10):
            print('  >> {:11s}: {:5f}'.format(classes[i], class_correct[i]/class_total[i]))
        torch.save(model.state_dict(), './model.pth')
        torch.save(optimizer.state_dict(), './optimizer.pth')

EPOCH  1:
	iteration  100: training_loss = 1.808029, test_loss = 1.535358
test_accuracy = 0.522800
  >> plane      : 0.448000
  >> car        : 0.866000
  >> bird       : 0.271000
  >> cat        : 0.329000
  >> deer       : 0.304000
  >> dog        : 0.501000
  >> frog       : 0.640000
  >> horse      : 0.713000
  >> ship       : 0.620000
  >> truck      : 0.536000
EPOCH  2:
	iteration  100: training_loss = 1.174139, test_loss = 1.059011
test_accuracy = 0.661100
  >> plane      : 0.677000
  >> car        : 0.885000
  >> bird       : 0.359000
  >> cat        : 0.653000
  >> deer       : 0.514000
  >> dog        : 0.423000
  >> frog       : 0.780000
  >> horse      : 0.735000
  >> ship       : 0.839000
  >> truck      : 0.746000
EPOCH  3:
	iteration  100: training_loss = 0.875948, test_loss = 0.867437
test_accuracy = 0.727300
  >> plane      : 0.745000
  >> car        : 0.899000
  >> bird       : 0.501000
  >> cat        : 0.697000
  >> deer       : 0.644000
  >> dog        : 0.481000
 